In [32]:
import pandas as pd
import json
import numpy as np
#Converts a dataframe to a geojson Point output
def df_to_geojson_point(df, properties, lat, lon):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

#Converts a dataframe to a geojson LineString output
def df_to_geojson_line(df, ycoord, xcoord, aggregateBy):
    
    #create new dataframe, aggregate points per bear to a linestring:
    uniqueValues=[]
    for _, row in df.iterrows():
        uniqueValues.append(row[aggregateBy])
    
    
    uniqueValues=list(set(uniqueValues))    
    
    aggDFlist=[]
    for uniqueValue in uniqueValues:
        coords=[]
        for _, row in df.iterrows():
            if row[aggregateBy] == uniqueValue:
                currx = row[xcoord]
                curry = row[ycoord]
                #coords+=",["+str(currx)+","+str(curry)+"]"
                coords.append([row[xcoord],row[ycoord]])
        
        #coords="["+coords[1:]+"]"
        aggDFlist.append([uniqueValue,coords])
    
    
    
    aggr_df = pd.DataFrame(aggDFlist, columns=["bear_name", "coords"])  
    
    properties=["bear_name"]
    geojson = {"type":"FeatureCollection", "features":[]}
    for _, row in aggr_df.iterrows():
        feature = {"type":"Feature",
                   "properties":{},
                   "geometry":{"type":"LineString",
                               "coordinates": ""}}
        #print row["coords"]
        feature["geometry"]["coordinates"] = row["coords"]
        for prop in properties:
            feature["properties"][prop] = row[prop]
        geojson["features"].append(feature)
    return str(geojson).replace("'",'"')



In [33]:
from IPython.display import Javascript
from shapely import geometry

#Create a javascript variable with our geojson data to visualize in the browser
#The data object 'vizObj' will be a global varialbe in our window that
#We can pass to another javascript function call

#-------------------------------------------------------------------------------
csvfile=r'./Data/Brown bear Slovenia 1993-1999UTM.csv'
df = pd.read_csv(csvfile, low_memory=False)

properties=[]
properties.append('timestamp')
properties.append('time_index')
properties.append('individual-local-identifier')

#create time index
def createTimeIndex (row):
   index = row['timestamp'][:4]+row['timestamp'][5:7]+row['timestamp'][8:10]
   return index
df['time_index'] = df.apply(lambda row: createTimeIndex (row),axis=1)
#print df.head(10)

#compute speed between two vertices of each bears trajectory:

#get unique bear names:
bears = set(list(df['individual-local-identifier']))

geojson = {"type":"FeatureCollection", "features":[]}
for bear in bears:
    df_subset = df[df['individual-local-identifier'] == bear]
    for i in range(1, len(df_subset)):
        currx = df_subset.iloc[i]['utm-easting']
        curry = df_subset.iloc[i]['utm-northing']
        prevx = df_subset.iloc[i-1]['utm-easting']
        prevy = df_subset.iloc[i-1]['utm-northing']   
        dist_to_prev = np.sqrt((currx-prevx)**2+(curry-prevy)**2) / 1000.0 #in km
        time_to_prev = (pd.Timestamp(df_subset.iloc[i]['timestamp']) - pd.Timestamp(df_subset.iloc[i-1]['timestamp']))/ pd.Timedelta('1 hour') #in hrs.
        if time_to_prev!=0:
            speed_from_prev = float(dist_to_prev)/float(time_to_prev)
        else:
            speed_from_prev = 0.0

        feature = {"type":"Feature",
                   "properties":{},
                   "geometry":{"type":"LineString",
                               "coordinates": ""}}          
        
        currx_geog = df_subset.iloc[i]['location-long']
        curry_geog = df_subset.iloc[i]['location-lat']
        prevx_geog = df_subset.iloc[i-1]['location-long']
        prevy_geog = df_subset.iloc[i-1]['location-lat']          

        feature["geometry"]["coordinates"] = [[prevx_geog,prevy_geog],[currx_geog,curry_geog]]
        feature["properties"]["bear_name"] = bear
        feature["properties"]["speed_from_prev_kmh"] = speed_from_prev
        feature["properties"]["dist_to_prev_km"] = dist_to_prev
        feature["properties"]["time_to_prev_h"] = time_to_prev
        
        geojson["features"].append(feature)        
geojson = str(geojson).replace("'",'"')                                         

Javascript("""window.vizObj_lines_speed={};""".format(geojson))


<IPython.core.display.Javascript object>

In [34]:
#Now let's make some HTML to style our intended mapbox output
from IPython.display import HTML
HTML("""

<link rel="stylesheet" href="https://unpkg.com/leaflet@1.0.1/dist/leaflet.css" />
<style> 
#mapid { height: 500px; }
</style>

""")

In [35]:
%%javascript
require.config({
  paths: {
      leaflet: "https://unpkg.com/leaflet@1.0.1/dist/leaflet.js"
  }
});

<IPython.core.display.Javascript object>

In [36]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;
require(['https://unpkg.com/leaflet@1.0.1/dist/leaflet.js'], function(leaflet){
//require(['leaflet'], function(leaflet){ //not working, why???
    
    var mymap = L.map('mapid').setView([45.86,14.30], 8);
    var OpenStreetMap_Mapnik = L.tileLayer('http://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', {
        maxZoom: 19,
        attribution: '&copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>' });     
    OpenStreetMap_Mapnik.addTo(mymap); 
    
    var CartoDB_DarkMatter = L.tileLayer('http://{s}.basemaps.cartocdn.com/dark_all/{z}/{x}/{y}.png', {
        attribution: '&copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> &copy; <a href="http://cartodb.com/attributions">CartoDB</a>',
        subdomains: 'abcd',
        maxZoom: 19
    }); 
    //CartoDB_DarkMatter.addTo(mymap); //alternative basemap from http://leaflet-extras.github.io/leaflet-providers/preview/
    
    var NASAGIBS_ViirsEarthAtNight2012 = L.tileLayer('http://map1.vis.earthdata.nasa.gov/wmts-webmerc/VIIRS_CityLights_2012/default/{time}/{tilematrixset}{maxZoom}/{z}/{y}/{x}.{format}', {
        attribution: 'Imagery provided by services from the Global Imagery Browse Services (GIBS), operated by the NASA/GSFC/Earth Science Data and Information System (<a href="https://earthdata.nasa.gov">ESDIS</a>) with funding provided by NASA/HQ.',
        bounds: [[-85.0511287776, -179.999999975], [85.0511287776, 179.999999975]],
        minZoom: 1,
        maxZoom: 8,
        format: 'jpg',
        time: '',
        tilematrixset: 'GoogleMapsCompatible_Level'
    });
    //NASAGIBS_ViirsEarthAtNight2012.addTo(mymap); //alternative basemap from http://leaflet-extras.github.io/leaflet-providers/preview/

    var OpenStreetMap_BlackAndWhite = L.tileLayer('http://{s}.tiles.wmflabs.org/bw-mapnik/{z}/{x}/{y}.png', {
        maxZoom: 18,
        attribution: '&copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>'
    });
    //OpenStreetMap_BlackAndWhite.addTo(mymap); //alternative basemap from http://leaflet-extras.github.io/leaflet-providers/preview/

    
    
    //var myLayer = L.geoJSON().addTo(mymap);
    //myLayer.addData(window.vizObj_lines_speed);
    //console.log(window.vizObj_lines_speed.features[0].properties.speed_from_prev_kmh)
    
   
    
    

    L.geoJSON(window.vizObj_lines_speed, {
        style: function(feature) {
            if (feature.properties.speed_from_prev_kmh >=0.0 && feature.properties.speed_from_prev_kmh <0.1) {return {color: "#fef0d9"}};
            if (feature.properties.speed_from_prev_kmh >=0.1 && feature.properties.speed_from_prev_kmh <0.2) {return {color: "#fdd49e"}};
            if (feature.properties.speed_from_prev_kmh >=0.2 && feature.properties.speed_from_prev_kmh <0.3) {return {color: "#fdbb84"}};
            if (feature.properties.speed_from_prev_kmh >=0.3 && feature.properties.speed_from_prev_kmh <0.4) {return {color: "#fc8d59"}};
            if (feature.properties.speed_from_prev_kmh >=0.4 && feature.properties.speed_from_prev_kmh <0.5) {return {color: "#ef6548"}};
            if (feature.properties.speed_from_prev_kmh >=0.5 && feature.properties.speed_from_prev_kmh <0.6) {return {color: "#d7301f"}};
            if (feature.properties.speed_from_prev_kmh >=0.6 && feature.properties.speed_from_prev_kmh <0.7) {return {color: "#990000"}};         }
    }).addTo(mymap);   //feature.properties.speed_from_prev not defined         
  


});
element.append('<div id="mapid"></div>');

<IPython.core.display.Javascript object>